<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Summary" data-toc-modified-id="Summary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></span></li><li><span><a href="#Reading-In-Data" data-toc-modified-id="Reading-In-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reading In Data</a></span></li><li><span><a href="#Analyzing-Data" data-toc-modified-id="Analyzing-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analyzing Data</a></span><ul class="toc-item"><li><span><a href="#High-Level-.describe()" data-toc-modified-id="High-Level-.describe()-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>High Level .describe()</a></span></li><li><span><a href="#Top-N-by-CV-ROC" data-toc-modified-id="Top-N-by-CV-ROC-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Top-N by CV-ROC</a></span></li><li><span><a href="#Finding-Maximum-CV-ROC-by-weight-scheme" data-toc-modified-id="Finding-Maximum-CV-ROC-by-weight-scheme-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Finding Maximum CV-ROC by weight scheme</a></span></li></ul></li></ul></div>

## Summary 

In this notebook we need to analyze the received experimental data and arrive to some inetertsing results and explanations that we'll add to the paper 

The metrics to consider for any analysis: 

 * "best" by ROC on CV and/or test (sometimes ROC si ste to 0 or -1 as an error, you should ignore those results)
 * another way to consider is "best" by F1: we have F1_fake and F1-real separately (more interetsed in fake actually but better to analyzie for both - see how Veronica and Rada did it in their paper).


What we might want to learn: 

* which experiment is the best overall?  

* which weight scheme gave the best results more frequently?  That is, may be the best weighting scheme. 
    
* which classifier performed better?  

* which feature combinations are better to use? 

* what are the feature vector sizes for what combinations? Any characteristics for those. Which potentially overfit and cannot be used on our dataset?   (might be useful in the future for larger datasets) 

* ... so on

As the result of ths investigatin you should be able to  provide teh following recommenation to the reader (very rough sample): 

we prepared this dataset for the Urdu language. Urdu doesn't have such tools as other languages taht's why we didn't use the facy features as in (Veronica and Rada, 2018) yet we tried to provide a baseline classifier and recommend the best set of readily available feature, i.e., N-grams of different types, as well as analyzed various hyperparameters such as weighting schemes for those features. We've tried our experiments using a number of popular and available classifiers and  found out that: 

* the best result was with such  n-gram combination - weighting scheme - classifier  (actually, by 2 or 3 metrics, we might get a few "bests")
* the overall consistently strongly performing weighting scheme is .... 
    * (also, logent and rela give aweful times but it might be because they are manually written by us - we'll see whether to add this) 
 
* the recommended classifier is.. 

* the promising n-gram combinations are ... (we need to exclude the combinatios with overly large feature #)



In [2]:
# IGNORE THIS: needed to make it run on Alisa's laptop 
import sys 
#sys.executable, sys.path
sys.path.append('/Users/alisa/workspace/Urdu_Fake_News/envname/lib/python3.7/site-packages')

In [3]:
import pandas as pd 
import numpy as np

## Reading In Data

In [5]:
results = pd.read_csv('results/results.csv')
results.shape

(2880, 16)

In [6]:
results.head()

,N-gram-feat-comb,char N-gram,word N-gram,func N-gram,total_feat_num,weight scheme,classifier,cv-acc,cv-balanced_acc,cv-roc,cv-f1-fake,acc,balanced_acc,roc,f1_fake,f1_real
0,1-0-0 char-word-func,1,0,0,101,binary,LogisticRegression,0.66,0.64,0.79,0.52,0.75,0.72,0.85,0.65,0.80
1,1-0-0 char-word-func,1,0,0,101,binary,LinearSVC,0.73,0.73,0.81,0.72,0.80,0.79,0.00,0.76,0.83
2,1-0-0 char-word-func,1,0,0,101,binary,SVC,0.67,0.64,0.79,0.46,0.77,0.75,0.00,0.68,0.82
3,1-0-0 char-word-func,1,0,0,101,binary,DecisionTreeClassifier,0.72,0.72,0.72,0.68,0.77,0.76,0.78,0.72,0.80
4,1-0-0 char-word-func,1,0,0,101,binary,RandomForestClassifier,0.65,0.61,0.80,0.32,0.72,0.68,0.89,0.52,0.80


## Analyzing Data

### High Level .describe()

In [9]:
results.describe()

,char N-gram,word N-gram,func N-gram,total_feat_num,cv-acc,cv-balanced_acc,cv-roc,cv-f1-fake,acc,balanced_acc,roc,f1_fake,f1_real
count,2880.000000,2880.000000,2880.000000,2880.00000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000
mean,1.600000,1.600000,1.600000,9049.80000,0.484639,0.468149,0.554896,0.291660,0.661969,0.629722,0.563872,0.425931,0.741056
std,1.925605,1.925605,1.925605,11885.41162,0.436200,0.433719,0.460627,0.460304,0.088442,0.108911,0.339367,0.296325,0.071564
min,0.000000,0.000000,0.000000,18.00000,-1.000000,-1.000000,-1.000000,-1.000000,0.480000,0.490000,0.000000,0.000000,0.170000
25%,0.000000,0.000000,0.000000,922.75000,0.550000,0.500000,0.590000,0.010000,0.580000,0.510000,0.382500,0.070000,0.720000
50%,1.000000,1.000000,1.000000,4079.50000,0.580000,0.570000,0.670000,0.490000,0.640000,0.630000,0.710000,0.540000,0.730000
75%,3.000000,3.000000,3.000000,9027.00000,0.650000,0.640000,0.750000,0.620000,0.730000,0.720000,0.810000,0.680000,0.790000
max,6.000000,6.000000,6.000000,41125.00000,0.830000,0.830000,0.910000,0.820000,0.870000,0.870000,0.960000,0.850000,0.900000


Look at the results and anayze each column.   What can we tell from this very high level analysis? 
Note that values -1 and 0 for ROC are when error occured or this metric could not eb applied (not all classifiers support it, do you know why?) 

For other metrics if the value is 0, so it is. 


### Top-N by CV-ROC

In [15]:
# https://www.geeksforgeeks.org/get-n-largest-values-from-a-particular-column-in-pandas-dataframe/
results.nlargest(5, ['cv-roc']) 

,N-gram-feat-comb,char N-gram,word N-gram,func N-gram,total_feat_num,weight scheme,classifier,cv-acc,cv-balanced_acc,cv-roc,cv-f1-fake,acc,balanced_acc,roc,f1_fake,f1_real
249,2-0-1 char-word-func,2,0,1,1848,binary,LinearSVC,0.82,0.82,0.91,0.81,0.84,0.84,0.0,0.81,0.87
250,2-0-1 char-word-func,2,0,1,1848,binary,SVC,0.83,0.83,0.91,0.82,0.84,0.83,0.0,0.80,0.87
9,2-0-0 char-word-func,2,0,0,1555,binary,LinearSVC,0.80,0.80,0.90,0.79,0.84,0.83,0.0,0.80,0.87
10,2-0-0 char-word-func,2,0,0,1555,binary,SVC,0.80,0.81,0.90,0.80,0.85,0.84,0.0,0.81,0.88
201,2-1-0 char-word-func,2,1,0,5575,binary,LinearSVC,0.80,0.80,0.90,0.78,0.87,0.86,0.0,0.84,0.89


We already kinda see that 2 - 0 - 1 , 2 - 0 -0 , or 2 - 1 -0 are teh best o teh Cross-Validation runs. Will this hold for tests? (actually, for test never mind, the function didn't apply there - then check the F-scores, whether they stay high  for  CV and Test runs. If they are sufficiently high (show up in top5, top 10), then we are on a good track).   


Note that the 1st column is the # of the line in the result table and can be tretaed as the ID of an experiment 

### Finding Maximum CV-ROC by weight scheme

In [14]:
# https://stackoverflow.com/questions/30880511/pandas-groupby-category-rating-get-top-value-from-each-category
results.loc[results.groupby('weight scheme')["cv-roc"].agg('idxmax')]

,N-gram-feat-comb,char N-gram,word N-gram,func N-gram,total_feat_num,weight scheme,classifier,cv-acc,cv-balanced_acc,cv-roc,cv-f1-fake,acc,balanced_acc,roc,f1_fake,f1_real
249,2-0-1 char-word-func,2,0,1,1848,binary,LinearSVC,0.82,0.82,0.91,0.81,0.84,0.84,0.00,0.81,0.87
730,2-0-1 char-word-func,2,0,1,1848,logent,SVC,0.55,0.50,0.90,0.00,0.57,0.50,0.00,0.00,0.73
1599,2-2-2 char-word-func,2,2,2,11448,none (tf),AdaBoostClassifier,0.81,0.80,0.89,0.78,0.85,0.84,0.93,0.82,0.88
1583,2-2-2 char-word-func,2,2,2,11448,norm,AdaBoostClassifier,0.81,0.80,0.89,0.78,0.85,0.84,0.93,0.82,0.88
1943,1-0-1 char-word-func,1,0,1,394,relat,AdaBoostClassifier,0.75,0.75,0.83,0.74,0.85,0.84,0.93,0.81,0.87
996,2-0-0 char-word-func,2,0,0,1555,tfidf,RandomForestClassifier,0.66,0.62,0.87,0.34,0.74,0.70,0.94,0.56,0.82
